In [1]:
import pandas as pd
import numpy as np


#visualization
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBaclend.figure_format = 'svg'


#preprocessing tools

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer #equal to get dummies
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder


In [2]:
diamonds = pd.read_csv('./data/diamonds_train.csv.zip')
diamonds_predict = pd.read_csv('./data/diamonds_predict.csv')

In [3]:
TARGET = 'price' 


NUM_FEATS = ['carat', 'depth', 'x', 'y']
CAT_FEATS = ['cut', 'color', 'clarity']
FEATS = NUM_FEATS + CAT_FEATS

In [4]:
numeric_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
               ('scaler', StandardScaler())])

In [5]:
categorical_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
               ('onehot', OneHotEncoder(handle_unknown='ignore'))])#handle unkown dont compute unkown values

In [6]:
preprocessor = \
ColumnTransformer(transformers=[('num', numeric_transformer, NUM_FEATS),
                               ('cat', categorical_transformer, CAT_FEATS)])

In [7]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['carat', 'depth', 'x', 'y']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['cut', 'color', 'clarity'])])

In [8]:
pd.DataFrame(data=preprocessor.fit_transform(diamonds).todense()).head()

#.todense(), to be able to see the dataframe properly and not tabular manner

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.867006,0.452019,0.978807,0.921985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-1.004557,0.871099,-1.226738,-1.179816,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.184434,2.617265,-0.097286,-0.176882,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.815298,1.429872,-0.933258,-0.883296,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.467458,-0.875068,0.729794,0.677793,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
diamonds_train, diamonds_test = train_test_split(diamonds)

In [11]:
print(diamonds_train.shape)
print(diamonds_test.shape)

(30341, 10)
(10114, 10)


In [12]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

model = Pipeline(steps=[('preprocessor',preprocessor),
                       ('regressor', RandomForestRegressor())])

In [13]:
model.fit(diamonds_train[FEATS], diamonds_train[TARGET]);

In [14]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [15]:
y_test = model.predict(diamonds_test[FEATS])
y_train = model.predict(diamonds_train[FEATS])


In [16]:
print(f"Test error: {mean_squared_error(y_pred=y_test,y_true=diamonds_test[TARGET], squared=False)}")
print(f"Train error: {mean_squared_error(y_pred=y_train,y_true=diamonds_train[TARGET], squared=False)}")

Test error: 556.8102165589221
Train error: 215.7900174551278


In [17]:
print(f"test mean sq error: {r2_score(y_pred=y_test, y_true=diamonds_test[TARGET])}")
print(f"train mean sq error: {r2_score(y_pred=y_train, y_true=diamonds_train[TARGET])}")

test mean sq error: 0.980390565562541
train mean sq error: 0.9970862796987879


In [19]:
scores = cross_val_score(model,
                        diamonds[FEATS],
                        diamonds[TARGET],
                        scoring='neg_root_mean_squared_error',
                        cv=5, n_jobs=-1)

In [20]:
np.mean(-scores)

567.1890840435052

In [21]:
from sklearn.model_selection import RandomizedSearchCV 

In [22]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'regressor__n_estimators': [16, 32, 64, 128, 256, 512],
    'regressor__max_depth': [2, 4, 8, 16],
}

grid_search = RandomizedSearchCV(model, 
                                 param_grid, 
                                 cv=5, 
                                 verbose=10, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1,
                                 n_iter=32)

grid_search.fit(diamonds[FEATS], diamonds[TARGET])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   43.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.2min
/home/emmanuel/miniconda3/envs/ironhack_env/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 14.6min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 28.6min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 34.2min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 35.9min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 37.

KeyboardInterrupt: 

In [24]:
grid_search.best_params_

{'regressor__n_estimators': 512,
 'regressor__max_depth': 16,
 'preprocessor__num__imputer__strategy': 'mean'}

In [25]:
grid_search.best_score_

-560.3655391805563

In [32]:
y_pred = grid_search.predict(diamonds_predict[FEATS])

IndexError: list index out of range

In [31]:
submission_df = pd.DataFrame({'id': diamonds_predict['id'],
                             'price': y_pred})

NameError: name 'y_pred' is not defined

In [ ]:
submission_df.head()

In [ ]:
submission_df.price.clip(350,20000, inplace=True)

In [ ]:
submission_df.to_csv('./submissions/submission_pipeline.ridge.csv', index=False)